In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd

FIRST READ DATA

In [2]:
df_counties_original = pd.read_csv("./us-counties.csv", dtype={"date": np.string_, 
                                                             "county": np.string_,
                                                             "state": np.string_,
                                                             "fips": np.string_,
                                                            "cases": np.int32,
                                                            "deaths": np.int32})
#Tansform Unknown fips to '00000' 
df_counties_original['fips'] = df_counties_original['fips'].replace({np.nan: '00000'})
df_counties_original.sort_values("fips", ascending=True)


,date,county,state,fips,cases,deaths
9971,2020-03-24,Unknown,Kentucky,00000,18,0
47027,2020-04-10,Unknown,Michigan,00000,433,7
46956,2020-04-10,Unknown,Massachusetts,00000,129,9
46915,2020-04-10,Unknown,Maine,00000,3,0
46894,2020-04-10,Unknown,Louisiana,00000,54,2
46739,2020-04-10,Unknown,Kansas,00000,0,4
46505,2020-04-10,Unknown,Illinois,00000,81,1
728,2020-03-07,Unknown,Rhode Island,00000,3,0
46427,2020-04-10,Unknown,Idaho,00000,3,0
46396,2020-04-10,Unknown,Hawaii,00000,4,0


Generate GeoJSON

In [3]:
pivot_counties = pd.pivot_table(df_counties_original , index=['state','county','fips'],
                       columns=['date'])
pivot_counties = pivot_counties.replace(np.nan, 0)
pivot_counties

cases                                   \
date                        2020-01-21 2020-01-22 2020-01-23 2020-01-24   
state     county      fips                                                
Alabama   Autauga     01001        0.0        0.0        0.0        0.0   
          Baldwin     01003        0.0        0.0        0.0        0.0   
          Barbour     01005        0.0        0.0        0.0        0.0   
          Bibb        01007        0.0        0.0        0.0        0.0   
          Blount      01009        0.0        0.0        0.0        0.0   
          Bullock     01011        0.0        0.0        0.0        0.0   
          Butler      01013        0.0        0.0        0.0        0.0   
          Calhoun     01015        0.0        0.0        0.0        0.0   
          Chambers    01017        0.0        0.0        0.0        0.0   
          Cherokee    01019        0.0        0.0        0.0        0.0   
          Chilton     01021        0.0        0.0        0.0        0.0   
          Choctaw     01023        0.0        0.0        0.0        0.0   
          Clarke      01025        0.0        0.0        0.0        0.0   
          Clay        01027        0.0        0.0        0.0        0.0   
          Cleburne    01029        0.0        0.0        0.0        0.0   
          Coffee      01031        0.0        0.0        0.0        0.0   
          Colbert     01033        0.0        0.0        0.0        0.0   
          Conecuh     01035        0.0        0.0        0.0        0.0   
          Coosa       01037        0.0        0.0        0.0        0.0   
          Covington   01039        0.0        0.0        0.0        0.0   
          Crenshaw    01041        0.0        0.0        0.0        0.0   
          Cullman     01043        0.0        0.0        0.0        0.0   
          Dale        01045        0.0        0.0        0.0        0.0   
          Dallas      01047        0.0        0.0        0.0        0.0   
          DeKalb      01049        0.0        0.0        0.0        0.0   
          Elmore      01051        0.0        0.0        0.0        0.0   
          Escambia    01053        0.0        0.0        0.0        0.0   
          Etowah      01055        0.0        0.0        0.0        0.0   
          Fayette     01057        0.0        0.0        0.0        0.0   
          Franklin    01059        0.0        0.0        0.0        0.0   
...                                ...        ...        ...        ...   
Wisconsin St. Croix   55109        0.0        0.0        0.0        0.0   
          Trempealeau 55121        0.0        0.0        0.0        0.0   
          Vilas       55125        0.0        0.0        0.0        0.0   
          Walworth    55127        0.0        0.0        0.0        0.0   
          Washington  55131        0.0        0.0        0.0        0.0   
          Waukesha    55133        0.0        0.0        0.0        0.0   
          Waupaca     55135        0.0        0.0        0.0        0.0   
          Waushara    55137        0.0        0.0        0.0        0.0   
          Winnebago   55139        0.0        0.0        0.0        0.0   
          Wood        55141        0.0        0.0        0.0        0.0   
Wyoming   Albany      56001        0.0        0.0        0.0        0.0   
          Campbell    56005        0.0        0.0        0.0        0.0   
          Carbon      56007        0.0        0.0        0.0        0.0   
          Converse    56009        0.0        0.0        0.0        0.0   
          Crook       56011        0.0        0.0        0.0        0.0   
          Fremont     56013        0.0        0.0        0.0        0.0   
          Goshen      56015        0.0        0.0        0.0        0.0   
          Hot Springs 56017        0.0        0.0        0.0        0.0   
          Johnson     56019        0.0        0.0        0.0        0.0   
          Laramie     56021        0.0        0.0        0

Get Date Info

In [4]:
date= pivot_counties['cases'].columns
date

Index(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
       '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04',
       '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09',
       '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14',
       '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19',
       '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24',
       '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
       '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
       '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10',
       '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15',
       '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20',
       '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25',

In [5]:
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [6]:
dt_start = datetime.strptime(dt_str_start, "%Y-%m-%d")
dt_end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=dt_start,end=dt_end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

82 DatetimeIndex(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
               '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
               '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01',
               '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05',
               '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09',
               '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13',
               '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17',
               '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21',
               '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25',
               '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
               '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
               '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12',
               '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16',
   

Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [7]:
# import json
# pivot_counties['cases_ts'] = json.dumps({"values": pivot_counties['cases'].values.tolist()[0]})
# pivot_counties['deaths_ts'] =  json.dumps({"values": pivot_counties['deaths'].values.tolist()[0]})

pivot_counties['cases_ts'] =  pivot_counties['cases'].values.tolist()
pivot_counties['cases_ts'] = pivot_counties['cases_ts'].apply(lambda x: ','.join(map(str, x)))
pivot_counties['deaths_ts'] =  pivot_counties['deaths'].values.tolist()
pivot_counties['deaths_ts'] = pivot_counties['deaths_ts'].apply(lambda x: ','.join(map(str, x)))

pivot_counties

cases                                   \
date                        2020-01-21 2020-01-22 2020-01-23 2020-01-24   
state     county      fips                                                
Alabama   Autauga     01001        0.0        0.0        0.0        0.0   
          Baldwin     01003        0.0        0.0        0.0        0.0   
          Barbour     01005        0.0        0.0        0.0        0.0   
          Bibb        01007        0.0        0.0        0.0        0.0   
          Blount      01009        0.0        0.0        0.0        0.0   
          Bullock     01011        0.0        0.0        0.0        0.0   
          Butler      01013        0.0        0.0        0.0        0.0   
          Calhoun     01015        0.0        0.0        0.0        0.0   
          Chambers    01017        0.0        0.0        0.0        0.0   
          Cherokee    01019        0.0        0.0        0.0        0.0   
          Chilton     01021        0.0        0.0        0.0        0.0   
          Choctaw     01023        0.0        0.0        0.0        0.0   
          Clarke      01025        0.0        0.0        0.0        0.0   
          Clay        01027        0.0        0.0        0.0        0.0   
          Cleburne    01029        0.0        0.0        0.0        0.0   
          Coffee      01031        0.0        0.0        0.0        0.0   
          Colbert     01033        0.0        0.0        0.0        0.0   
          Conecuh     01035        0.0        0.0        0.0        0.0   
          Coosa       01037        0.0        0.0        0.0        0.0   
          Covington   01039        0.0        0.0        0.0        0.0   
          Crenshaw    01041        0.0        0.0        0.0        0.0   
          Cullman     01043        0.0        0.0        0.0        0.0   
          Dale        01045        0.0        0.0        0.0        0.0   
          Dallas      01047        0.0        0.0        0.0        0.0   
          DeKalb      01049        0.0        0.0        0.0        0.0   
          Elmore      01051        0.0        0.0        0.0        0.0   
          Escambia    01053        0.0        0.0        0.0        0.0   
          Etowah      01055        0.0        0.0        0.0        0.0   
          Fayette     01057        0.0        0.0        0.0        0.0   
          Franklin    01059        0.0        0.0        0.0        0.0   
...                                ...        ...        ...        ...   
Wisconsin St. Croix   55109        0.0        0.0        0.0        0.0   
          Trempealeau 55121        0.0        0.0        0.0        0.0   
          Vilas       55125        0.0        0.0        0.0        0.0   
          Walworth    55127        0.0        0.0        0.0        0.0   
          Washington  55131        0.0        0.0        0.0        0.0   
          Waukesha    55133        0.0        0.0        0.0        0.0   
          Waupaca     55135        0.0        0.0        0.0        0.0   
          Waushara    55137        0.0        0.0        0.0        0.0   
          Winnebago   55139        0.0        0.0        0.0        0.0   
          Wood        55141        0.0        0.0        0.0        0.0   
Wyoming   Albany      56001        0.0        0.0        0.0        0.0   
          Campbell    56005        0.0        0.0        0.0        0.0   
          Carbon      56007        0.0        0.0        0.0        0.0   
          Converse    56009        0.0        0.0        0.0        0.0   
          Crook       56011        0.0        0.0        0.0        0.0   
          Fremont     56013        0.0        0.0        0.0        0.0   
          Goshen      56015        0.0        0.0        0.0        0.0   
          Hot Springs 56017        0.0        0.0        0.0        0.0   
          Johnson     56019        0.0        0.0        0.0        0.0   
          Laramie     56021        0.0        0.0        0

In [8]:
yesterday = date.values[-2]
pivot_counties['today_case'] = pivot_counties['cases'][dt_str_end]
pivot_counties['today_new_case'] = pivot_counties['cases'][dt_str_end] - pivot_counties['cases'][yesterday]
pivot_counties['today_death'] = pivot_counties['deaths'][dt_str_end]
pivot_counties['today_new_death'] = pivot_counties['deaths'][dt_str_end] - pivot_counties['deaths'][yesterday]

In [9]:
pivot_counties['dt_first_case'] = (pivot_counties['cases'] > 0).idxmax(axis=1)
pivot_counties['dt_first_death'] = (pivot_counties['deaths'] > 0).idxmax(axis=1)
#For death reports, deal with no deaths counties
pivot_counties.loc[pivot_counties['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_counties.head(5)


cases                                              \
date                  2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                                                                              \
date                  2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                       ...     deaths             \
date                   ... 2020-04-10 2020-04-11   
state   county  fips   ...                         
Alabama Autauga 01001  ...        1.0        1.0   
        Baldwin 01003  ...        1.0        1.0   
        Barbour 01005  ...        0.0        0.0   
        Bibb    01007  ...        0.0        0.0   
        Blount  01009  ...        0.0        0.0   

                                                                cases_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Baldwin 01003  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Barbour 01005  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Bibb    01007  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Blount  01009  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   

                                                               deaths_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Baldwin 01003  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Barbour 01005  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Bibb    01007  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Blount  01009  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   

                      today_case today_new_case today_death today_new_death  \
date                                                                          
state   county  fips                                                          
Alabama Autauga 01001       19.0            2.0         1.0             0.0   
        Baldwin 01003       66.0            7.0         1.0             0.0   
        Barbour 01005       10.0            1.0         0.0             0.0   
        Bibb    01007       13.0            2.0         0.0             0.0   
        Blount  01009       12.0            0.0         0.0             0.0   

                      dt_first_case dt_first_death  
date                                                
state   county  fips                                
Alabama Autauga 01001    2020-03-24     2020-04-06  
        Baldwin 01003    2020-03-14     2020-04-01  
        Barbour 01005    2020-04-03            NaN  
        Bibb    01007    2020-03-30            NaN  
        Blount  01009    2020-03-25            NaN  

[5 rows x 172 columns]

Read County Boundary GeoJSON file

In [10]:
old_counties_geojson_df = gpd.read_file(r"./counties_update_new.geojson")
old_counties_geojson_df.head(5)


,NAME,state_name,GEOID,population,geometry
0,Bladen,North Carolina,37017,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,37167,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,39153,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,42113,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,48459,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


Old Data Structure

In [11]:
report_df = pivot_counties[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['state','county','fips','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Alabama,Autauga,01001,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-24,2020-04-06,19.0,2.0,1.0,0.0
1,Alabama,Baldwin,01003,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-14,2020-04-01,66.0,7.0,1.0,0.0
2,Alabama,Barbour,01005,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-03,NaN,10.0,1.0,0.0,0.0
3,Alabama,Bibb,01007,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-30,NaN,13.0,2.0,0.0,0.0
4,Alabama,Blount,01009,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-25,NaN,12.0,0.0,0.0,0.0


In [13]:
report_df['today_case'].sum()
# Truth from report: 528422

528422.0

In [29]:
geojson_counties = (old_counties_geojson_df['state_name'] + "," + old_counties_geojson_df['NAME']).unique()
geojson_counties

array(['North Carolina,Bladen', 'North Carolina,Stanly', 'Ohio,Summit',
       ..., 'Georgia,Sumter', 'Missouri,Kansas City',
       'New York,New York City'], dtype=object)

In [30]:
report_counties = (report_df['state'] + "," + report_df['county']).unique()
report_counties

array(['Alabama,Autauga', 'Alabama,Baldwin', 'Alabama,Barbour', ...,
       'Wyoming,Teton', 'Wyoming,Uinta', 'Wyoming,Washakie'], dtype=object)

In [31]:
for county in geojson_counties:
    if county not in report_counties:
        print(county);

West Virginia,Grant
South Dakota,Haakon
Nebraska,Pawnee
Wisconsin,Forest
Hawaii,Kalawao
New York,Bronx
Oklahoma,Cimarron
North Dakota,Cavalier
North Dakota,Towner
Nebraska,Perkins
South Dakota,Perkins
Virginia,Patrick
Indiana,Pike
Nebraska,Sioux
Minnesota,Morrison
New Mexico,De Baca
Illinois,Brown
Nebraska,Saline
Oklahoma,Blaine
Puerto Rico,Vega Baja
Illinois,Union
Missouri,Hickory
Virginia,Winchester
Texas,Fisher
Texas,Reagan
Wyoming,Platte
Texas,Freestone
Kansas,Cheyenne
North Dakota,Renville
Utah,Piute
Virginia,Covington
Puerto Rico,Guaynabo
Puerto Rico,Sabana Grande
Illinois,Scott
California,Modoc
Iowa,Emmet
Kansas,Harper
Georgia,Evans
Puerto Rico,Toa Baja
South Dakota,Harding
North Carolina,Yancey
Tennessee,Van Buren
Puerto Rico,Trujillo Alto
Texas,Kimble
Kansas,Rush
Kansas,Kiowa
Nebraska,Greeley
Illinois,Henderson
Illinois,Johnson
Virginia,Staunton
Texas,Ochiltree
Oklahoma,Harper
Puerto Rico,Yabucoa
Kentucky,Elliott
Iowa,Adams
Illinois,Putnam
Alaska,Aleutians West
Kansas,Logan
So

In [20]:
for state in report_states:
    if state not in geojson_states:
        print(state);        

Guam
Northern Mariana Islands
Virgin Islands


In [21]:
report_df[(report_df['state'] == 'Guam')
          |(report_df['state'] == 'Northern Mariana Islands')
          |(report_df['state'] == 'Virgin Islands')]

,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
516,Guam,Unknown,00000,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-15,2020-03-22,681.0,104.0,5.0,1.0
1749,Northern Mariana Islands,Unknown,00000,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-28,2020-04-01,11.0,0.0,2.0,0.0
2401,Virgin Islands,Unknown,00000,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-14,2020-04-06,51.0,5.0,1.0,0.0


In [76]:
exception = report_df[(report_df['fips'] == '00000') & (report_df['county'] == 'Unknown')][['county','state','today_case']]

In [77]:
exception

,county,state,today_case
77,Unknown,Alaska,0.0
92,Unknown,Arizona,0.0
160,Unknown,Arkansas,19.0
216,Unknown,California,0.0
273,Unknown,Colorado,175.0
284,Unknown,Connecticut,456.0
289,Unknown,Delaware,13.0
354,Unknown,Florida,2.0
500,Unknown,Georgia,1300.0
516,Unknown,Guam,681.0


In [78]:
exception['today_case'].sum()
# cases which has no county level info: 6558

6558.0

In [56]:
report_df[report_df['county'] == 'Cass'][['county','state','today_case']]

,county,state,today_case
651,Cass,Indiana,23.0
1109,Cass,Michigan,12.0
1186,Cass,Minnesota,3.0
1342,Cass,Missouri,32.0
1453,Cass,Nebraska,3.0
1720,Cass,North Dakota,95.0
2204,Cass,Texas,5.0


In [12]:
final_df = pd.merge(old_counties_geojson_df[["NAME", "state_name", "population", "geometry"]], report_df, how='left', left_on=['NAME','state_name'], right_on = ['county','state'])
final_df.columns

Index(['NAME', 'state_name', 'population', 'geometry', 'state', 'county',
       'fips', 'cases_ts', 'deaths_ts', 'dt_first_case', 'dt_first_death',
       'today_case', 'today_new_case', 'today_death', 'today_new_death'],
      dtype='object')

In [13]:
final_df=final_df[["NAME", "state_name", "population", "fips", "dt_first_case", "dt_first_death", "cases_ts", "deaths_ts", 'today_case','today_new_case', 'today_death','today_new_death', "geometry"]]
final_df['dt_start'] = dt_str_start
final_df['dt_end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,dt_start,dt_end,dt_unit
0,Bladen,North Carolina,33778,37017,2020-03-29,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,0.0,0.0,0.0,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",2020-01-21,2020-04-11,day
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",14.0,3.0,1.0,0.0,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ...",2020-01-21,2020-04-11,day
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",243.0,8.0,14.0,1.0,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ...",2020-01-21,2020-04-11,day
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,0.0,0.0,0.0,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ...",2020-01-21,2020-04-11,day
4,Upshur,Texas,40769,48459,2020-03-20,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",6.0,0.0,0.0,0.0,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ...",2020-01-21,2020-04-11,day
5,Brown,Texas,37834,48049,2020-03-20,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",10.0,2.0,0.0,0.0,"MULTIPOLYGON (((-99.19587 32.07923, -99.11866 ...",2020-01-21,2020-04-11,day
6,Cherokee,South Carolina,56711,45021,2020-04-02,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",8.0,0.0,0.0,0.0,"MULTIPOLYGON (((-81.87441 35.18352, -81.76813 ...",2020-01-21,2020-04-11,day
7,Cullman,Alabama,82313,01043,2020-03-20,2020-04-01,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",31.0,1.0,1.0,0.0,"MULTIPOLYGON (((-87.11199 33.99238, -87.10991 ...",2020-01-21,2020-04-11,day
8,Grant,West Virginia,11641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-79.48687 39.20596, -79.42441 ...",2020-01-21,2020-04-11,day
9,Haakon,South Dakota,2082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-102.00107 44.51093, -102.0004...",2020-01-21,2020-04-11,day


In [14]:
final_df[pd.notnull(final_df["cases_ts"])]['dt_start'] = dt_str_start
final_df.head(5)

C:\Users\Jason\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,dt_start,dt_end,dt_unit
0,Bladen,North Carolina,33778,37017,2020-03-29,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,0.0,0.0,0.0,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",2020-01-21,2020-04-11,day
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",14.0,3.0,1.0,0.0,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ...",2020-01-21,2020-04-11,day
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",243.0,8.0,14.0,1.0,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ...",2020-01-21,2020-04-11,day
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,0.0,0.0,0.0,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ...",2020-01-21,2020-04-11,day
4,Upshur,Texas,40769,48459,2020-03-20,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",6.0,0.0,0.0,0.0,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ...",2020-01-21,2020-04-11,day


In [15]:
final_df[['today_case','today_new_case','today_death','today_new_death']] = final_df[['today_case','today_new_case','today_death','today_new_death']].replace(np.nan,0)


In [79]:
final_df['today_case'].sum()
# Truth total from case report: 528422
# Previous total without Kansas City: 522461
# Adding Kansas City: 522759 + Unknown cases: 6558 = 529317?

522759.0

In [82]:
final_df[final_df['NAME'] == 'Unknown']

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,dt_start,dt_end,dt_unit


In [16]:
final_df.to_file(r"./nyt_counties_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done


New Data Structure (Full version)

In [ ]:
# pivot_counties.columns = ['_'.join(col).strip() for col in pivot_counties.columns.values]
# report_df_2 = pivot_counties.reset_index()

In [ ]:
# report_df_2.head(5)

In [ ]:
# pivot_counties.head(5)

In [ ]:
# final_df_2 = pd.merge(old_counties_geojson_df[["NAME", "state_name", "population", "geometry"]], report_df_2, how='right', left_on=['NAME','state_name'], right_on = ['county','state'])
# final_df_2 = final_df_2.drop(['state','county'], axis = 1)
# final_df_2['dt_start'] = dt_start
# final_df_2['dt_end'] = dt_end
# final_df_2.head(5)

In [ ]:
# final_df_2.to_file(r"./nyt_counties_data_full.geojson", driver='GeoJSON', encoding='utf-8')
# print("done")

Generate JSON file

In [ ]:
# county_list = []
# for state in df_counties_original.state.unique():
#     for county in df_counties_original.loc[(df_counties_original['state'] == state)].county.unique():
#         county_list.append({"name": county, "state": state})
 
# print(len(county_list), county_list)


In [ ]:
# data_df = report_df.set_index(report_df["county"]+", "+report_df["state"])
# data_dict =data_df.to_dict(orient='index')

In [ ]:
# df = df_counties_original.groupby(['county', 'state'], sort=False)[['cases', 'deaths']].max().sort_values("cases", ascending=False)
# county_state_list = list(map(lambda x: x[0]+", "+x[1], df.index.tolist()))
# meta = {"dates": dt_range_str,"states": county_state_list, "cases": df["cases"].values.tolist(), "deaths": df["deaths"].values.tolist()}
# data_dict["metadata"] = meta

In [ ]:
# data_dict

In [ ]:
# import json
# with open('./nyt_counties_data.json', 'w') as outfile:
#     json.dump(data_dict, outfile)